# Idea
The idea is that we predict rotated linear dimension position from lines and text from the drawing.

# Thoughts
Basic logic is we split dataset to input lines or texts positions StartPoint, EndPoint, Position XYZ and predict dimension extension line poistion XLine1Point, XLine2Point XYZ.

We going to group samples by FileId. That is each sample will contain variable length data (attributes of variable number of  lines and text) and variable output data (variable number of dimensions).

I intend to [use RNN for it](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)

In [6]:
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
from processing import *

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
client = MongoClient('mongodb://192.168.0.104:27017/')
db = client.geometry

In [8]:
fileids  = db.files.find({'Valid':True}).distinct('FileId')
print(len(fileids))

query = {
    'ClassName':
    {
        '$in':[
            'AcDbLine',
            'AcDbText',
            'AcDbRotatedDimension'
        ]
    },
    'FileId':{
        '$in':fileids
    }
}

data = pd.DataFrame(list(db.objects.find(query)))

322


In [9]:
cols_to_expand = ['XLine1Point', 'XLine2Point','StartPoint','EndPoint','Position']
data = expand_columns(data, cols_to_expand)


In [12]:
len(data)

917991

In [15]:
test_data = data.sample(frac = 0.01)
test_data.groupby(['FileId', 'ClassName']).count()

_id  Color  Layer  \
FileId                               ClassName                                 
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                3      3      3   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                4      4      4   
                                     AcDbRotatedDimension    1      1      1   
                                     AcDbText                3      3      3   
0100c16b-c7c9-4265-92b9-950bf5d2c9ec AcDbLine                2      2      2   
...                                                        ...    ...    ...   
85f78f77-85c3-484a-964e-7769ea5804ae AcDbRotatedDimension    2      2      2   
864ce51e-79b8-4c65-99ca-dfdf738dfff2 AcDbLine                1      1      1   
                                     AcDbText                2      2      2   
87780373-9950-40b6-871c-6346f04b90af AcDbLine                6      6      6   
                                     AcDbText                2      2      2   

                                                           LineWeight  \
FileId                               ClassName                          
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                       3   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                       4   
                                     AcDbRotatedDimension           1   
                                     AcDbText                       3   
0100c16b-c7c9-4265-92b9-950bf5d2c9ec AcDbLine                       2   
...                                                               ...   
85f78f77-85c3-484a-964e-7769ea5804ae AcDbRotatedDimension           2   
864ce51e-79b8-4c65-99ca-dfdf738dfff2 AcDbLine                       1   
                                     AcDbText                       2   
87780373-9950-40b6-871c-6346f04b90af AcDbLine                       6   
                                     AcDbText                       2   

                                                           Linetype  ObjectId  \
FileId                               ClassName                                  
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                     3         3   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                     4         4   
                                     AcDbRotatedDimension         1         1   
                                     AcDbText                     3         3   
0100c16b-c7c9-4265-92b9-950bf5d2c9ec AcDbLine                     2         2   
...                                                             ...       ...   
85f78f77-85c3-484a-964e-7769ea5804ae AcDbRotatedDimension         2         2   
864ce51e-79b8-4c65-99ca-dfdf738dfff2 AcDbLine                     1         1   
                                     AcDbText                     2         2   
87780373-9950-40b6-871c-6346f04b90af AcDbLine                     6         6   
                                     AcDbText                     2         2   

                                                           EndPoint  Length  \
FileId                               ClassName                                
006cb2dd-6f18-4203-8b70-fc865c08105a AcDbLine                     3       3   
006f290c-7280-491b-b7d7-971ed82de1a5 AcDbLine                     4       4   
                                     AcDbRotatedDimension         0       0   
                                     AcDbText                     0       0   
0100c16b-c7c9-4265-92b9-950bf5d2c9ec AcDbLine                     2       2   
...                                                             ...     ...   
85f78f77-85c3-484a-964e-7769ea5804ae AcDbRotatedDimension         0       0   
864ce51e-79b8-4c65-99ca-dfdf738dfff2 AcDbLine                     1       1   
                                     AcDbText                     0       0   
87780373-9950-40b6-871c-6346f04b90af AcDbLine                     6       6   
                                     AcDbText                     

In [16]:
test_data.columns

Index(['_id', 'Color', 'Layer', 'LineWeight', 'Linetype', 'ObjectId',
       'ClassName', 'EndPoint', 'Length', 'StartPoint', 'FileId',
       'DimLinePoint', 'DimensionStyleName', 'DimensionText', 'TextPosition',
       'XLine1Point', 'XLine2Point', 'Position', 'TextString',
       'XLine1Point.ClassName', 'XLine1Point.X', 'XLine1Point.Y',
       'XLine1Point.Z', 'XLine2Point.ClassName', 'XLine2Point.X',
       'XLine2Point.Y', 'XLine2Point.Z', 'StartPoint.ClassName',
       'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z', 'EndPoint.ClassName',
       'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z', 'Position.ClassName',
       'Position.X', 'Position.Y', 'Position.Z'],
      dtype='object')

In [119]:
test_columns = [
    'ClassName', 'FileId',
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z',
                
   'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',

   'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z',
   'Position.X', 'Position.Y', 'Position.Z']

In [319]:
test_data['ClassName'] = test_data['ClassName'].astype('category')
test_data['FileId'] = test_data['FileId'].astype('category')
test_data[test_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9180 entries, 480468 to 11227
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   ClassName      9180 non-null   category
 1   FileId         9180 non-null   category
 2   XLine1Point.X  497 non-null    float64 
 3   XLine1Point.Y  497 non-null    float64 
 4   XLine1Point.Z  497 non-null    float64 
 5   XLine2Point.X  497 non-null    float64 
 6   XLine2Point.Y  497 non-null    float64 
 7   XLine2Point.Z  497 non-null    float64 
 8   StartPoint.X   7672 non-null   float64 
 9   StartPoint.Y   7672 non-null   float64 
 10  StartPoint.Z   7672 non-null   float64 
 11  EndPoint.X     7672 non-null   float64 
 12  EndPoint.Y     7672 non-null   float64 
 13  EndPoint.Z     7672 non-null   float64 
 14  Position.X     1011 non-null   float64 
 15  Position.Y     1011 non-null   float64 
 16  Position.Z     1011 non-null   float64 
dtypes: category(2), float64(15)

# Dataset implementation

In [320]:
# https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
import torch
from torch import nn
from torch.utils.data import Dataset, SubsetRandomSampler

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
device

device(type='cpu')

In [338]:
x_columns = [
    #'ClassName', 
    'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',
    'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z',
    'Position.X', 'Position.Y', 'Position.Z']

y_columns = [
    #'ClassName', 
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z']

class EntityDataset(Dataset):
    def __init__(self, pandasData):
        self.data = pandasData
        self.groupped = data.groupby('FileId')
        self.keys = list(self.groupped.groups.keys() )
        
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, index):
        x = self.groupped.get_group(self.keys[index])[x_columns]
        y = self.groupped.get_group(self.keys[index])[y_columns]
        
        return torch.tensor(np.array(x.values)), torch.tensor(np.array(y.values))

In [339]:
entities = EntityDataset(test_data)
c,d = entities[3]
print(c.shape, d.shape)

torch.Size([1265, 9]) torch.Size([1265, 6])


In [330]:
batch_size = 1

data_len = len(entities)

validation_fraction = 0.2
test_fraction       = 0.3

val_split  = int(np.floor(validation_fraction * data_len))
test_split = int(np.floor(test_fraction * data_len))
indices = list(range(data_len))
np.random.seed(228)

np.random.shuffle(indices)

val_indices   = indices[:val_split]
test_indices  = indices[val_split:test_split]
train_indices = indices[test_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler   = SubsetRandomSampler(val_indices)
test_sampler  = SubsetRandomSampler(test_indices)

# https://stackoverflow.com/questions/64586575/adding-class-objects-to-pytorch-dataloader-batch-must-contain-tensors
def custom_collate(sample):
    print(len(sample))
    x,y = sample[0]
    # print(x.shape,y.shape)
    
    return x,y

train_loader = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = train_sampler, collate_fn=custom_collate)
val_loader   = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = val_sampler)
test_loader  = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = test_sampler)

In [ ]:
test_data

In [331]:
(a,b) = next(iter(train_loader))

print(a.shape,b.shape)


1
torch.Size([472, 9]) torch.Size([472, 6])


# Model and training


In [334]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first = True)
        self.output_layer = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        # print('x:', x.shape)
        batch_size = x.shape[0]
        hidden = self.init_hidden(batch_size)
        #print(x.shape, hidden.shape)
        out, hidden = self.rnn(x, hidden)
        
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    def init_hidden(self, batch_size):
        # print(batch_size)
        hidden = torch.zeros(batch_size, self.n_layers,  self.hidden_dim)
        return hidden

In [352]:
# https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
# shape coordination
def test_rnn_shapes():
    in_size = 9
    out_size = 6
    hidden_size = 12
    batch_l = 65
    num_files = 166
    num_rnn_layers = 2
    rnn = torch.nn.RNN(input_size = in_size, hidden_size = hidden_size, num_layers = num_rnn_layers)
    inpt = torch.randn(num_files, batch_l, in_size)
    h0 = torch.randn(num_rnn_layers, batch_l, hidden_size)
    out, hn= rnn(inpt, h0)
    print ('out.shape', out.shape, 'hn.shape', hn.shape)
test_rnn_shapes()

out.shape torch.Size([166, 65, 12]) hn.shape torch.Size([2, 65, 12])


In [350]:

for idx, (inp, outp) in enumerate(train_loader):
    print("input shape:", inp.shape)
    out, hidden = rnn(inp)
    print("output shape", out.shape)
    break

1
input shape: torch.Size([671, 9])


RuntimeError: input must have 3 dimensions, got 2

In [280]:
def plot_history(loss_history, train_history, val_history):
    plt.ylabel('Accuracy @ epoch')
    
    train, = plt.plot(train_history)
    train.set_label("train")
    validation, = plt.plot(val_history)
    validation.set_label("validation")

    plt.legend()
    plt.show()

In [84]:
def compute_accuracy(model, loader):
    pass

def train_model(model, train_loader, val_loader, loss, optimizer, epochs):
    start = time.time()
    
    loss_history = []
    train_istory = []
    val_history  = []
    
    for epoch in range(epochs):
        model.train()
        
        loss_accumulated = 0
        
        i_step = 1
        for (x,y) in enumerate(train_loader):
            optimizer.zero_grad()
            
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            output, hidden = model(x_gpu)
            loss_value = loss(output, y_gpu)
            
            loss_value.backward()
            optimizer.step()
            
            loss_accumulated += loss_value
            
            #val_accuracy = compute_accuracy(model, val_loader)
            print('[{0} @ {4} sec] Loss: {1:4f} Train err: {2:2.2f}% Val err: {3:2.2f}%'. format(
                epoch,
                ave_loss,
                (1 - train_accuracy) * 100,
                (1 - val_accuracy) * 100,
                time.time() - start
            ))
        
        # loss_history.append(float(loss_accum / i_step))
        # val_history.append(val_accuracy)
        # clear_output(wait=True)
        
        i_step += 1
        
    plot_history(loss_history, train_history, val_history)

In [ ]:
lr = 1e-3
epochs = 3
optimizer = optim.Adam(lr = lr)
loss = nn.MSELoss()
model = RNN()

model.to(device)
train_model(model,train_loader, val_loader, loss, optimizer, epochs)

In [ ]:
torch.cuda.empty_cache()
pre
model.eval()
for (x,y) in test_loader:
    with torch.no_grad():
        output = model.forward(x.to(device))
        accuracy = compute_accuracy(model, test_loader)